In [ ]:
# Install the necessary packages
!pip install langchain -qU
!pip install langchain-openai -qU
!pip install langchain-community -qU

!pip install duckduckgo-search -qU
!pip install pyowm -qU

!pip install langchainhub -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 34.7 MB/s eta 0:00:00


In [ ]:
from langchain import hub  # Pull the prompt template from LangChainHub

# Fetch the prompt template
prompt = hub.pull("hwchase17/react")

# Convert the prompt template to a string
prompt_str = str(prompt)

# Split the prompt string into multiple lines for better readability
formatted_prompt = "\n".join([prompt_str[i:i+80] for i in range(0, len(prompt_str), 80)])

# Save the formatted prompt string to a text file
with open("prompt_template.txt", "w") as file:
    file.write(formatted_prompt)

print("Prompt saved to 'prompt_template.txt' with improved readability.")


Prompt saved to 'prompt_template.txt' with improved readability.


/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
import os
from google.colab import userdata

In [ ]:
from langchain_openai import ChatOpenAI

# Set OpenAI API key
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Initialize the ChatOpenAI model
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

# Initialize DuckDuckGo search tool
search_tool = DuckDuckGoSearchRun()

search_tool

DuckDuckGoSearchRun(api_wrapper=DuckDuckGoSearchAPIWrapper(region='wt-wt', safesearch='moderate', time='y', max_results=5, backend='auto', source='text'))

In [ ]:
from langchain_community.utilities import OpenWeatherMapAPIWrapper
from langchain.agents import load_tools

os.environ["OPENWEATHERMAP_API_KEY"] = userdata.get('OPENWEATHERMAP_API_KEY')

# Initialize OpenWeatherMap tool
weather = OpenWeatherMapAPIWrapper()

weather_tool = load_tools(["openweathermap-api"], llm)[0]

weather_tool


OpenWeatherMapQueryRun(api_wrapper=OpenWeatherMapAPIWrapper(owm=<pyowm.owm.OWM - API key=************************5d2ee3f8, subscription type=free, PyOWM version=(3, 3, 0)>, openweathermap_api_key=None))

In [ ]:
from langchain.agents import create_react_agent

# List of tools for the agent
tools = [search_tool, weather_tool]

# Create the agent using the LLM and the prompt template
agent = create_react_agent(tools = tools, llm = llm, prompt=prompt)

In [ ]:
from langchain.agents import AgentExecutor

# Initialize the agent executor with the created agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)


In [ ]:
from langchain.agents import tool

@tool
def word_count(text: str) -> int:
  """Returns the word count."""
  return len(text.split())

In [ ]:
# Add the custom tool to the list of tools
tools.append(word_count)

In [ ]:
import pandas as pd
from langchain.tools import tool

@tool
def flight(origin: str, destination: str) -> str:
    """
      Description: A flight information retrieval tool.
      Parameters:
      origin: The city you'll be flying out from.
      destination: The city you aim to reach.
      Example: FlightSearch[New York, London] would fetch flights from New York to London .
    """

    # Path to your CSV file
    path = "/content/clean_Flights_2022.csv"

    # Load the flight data
    data = pd.read_csv(path).dropna()[[
        'Flight Number', 'Price', 'DepTime', 'ArrTime', 'ActualElapsedTime',
        'FlightDate', 'OriginCityName', 'DestCityName', 'Distance'
    ]]

    # Filter the data based on input parameters
    results = data[
        (data["OriginCityName"] == origin) &
        (data["DestCityName"] == destination)
    ]

    # If no flights are found, return a message
    if results.empty:
        return f"There are no flights from {origin} to {destination} ."

    # Format the results into a readable string
    formatted_results = results.to_string(index=False)
    return f"Available flights:\n{formatted_results}"



In [ ]:
import pandas as pd
from langchain.tools import tool

@tool
def flight(origin: str, destination: str) -> str:
    """
    Search for flights by origin, destination, and departure date from a CSV file.

    Parameters:
    - origin (str): The origin city name.
    - destination (str): The destination city name.


    Returns:
    - Union[str, DataFrame]: Returns a DataFrame with flight details or a string if no flights are found.
    """

    # Path to your CSV file
    path = "/content/clean_Flights_2022.csv"

    # Load the flight data
    data = pd.read_csv(path).dropna()[[
        'Flight Number', 'Price', 'DepTime', 'ArrTime', 'ActualElapsedTime',
        'FlightDate', 'OriginCityName', 'DestCityName', 'Distance'
    ]]

    # Filter the data based on input parameters
    results = data[
        (data["OriginCityName"] == origin) &
        (data["DestCityName"] == destination)
    ]

    # If no flights are found, return a message
    if results.empty:
        return f"There are no flights from {origin} to {destination} ."

    # Format the results into a readable string
    formatted_results = results.to_string(index=False)
    return f"Available flights:\n{formatted_results}"



In [ ]:
# Add the custom tool to the list of tools
tools.append(flight)

In [ ]:
# Create a new agent with the DuckDuckGo search tool, OpenWeatherMap tool  and custom word count tool
agent = create_react_agent(tools = tools, llm = llm, prompt=prompt)

# Initialize the agent executor with the new agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)


In [ ]:
response = agent_executor.invoke({"input": "what is the word count of 'dushan fernando'"})

print(response['output'])



> Entering new AgentExecutor chain...
I need to determine the word count of the given text. 
Action: word_count
Action Input: 'dushan fernando'2I now know the final answer
Final Answer: 2

> Finished chain.
2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data

NameError: name 'data' is not defined

In [ ]:
def u(origin: str, destination: str):
    try:
          # Path to your CSV file
          path = "/content/clean_Flights_2022.csv"

          # Load the flight data
          data = pd.read_csv(path).dropna()[[
              'Flight Number', 'Price', 'DepTime', 'ArrTime', 'ActualElapsedTime',
              'FlightDate', 'OriginCityName', 'DestCityName', 'Distance'
          ]]

          # Filter the data based on input parameters
          results = data[
              (data["OriginCityName"] == origin) &
              (data["DestCityName"] == destination)
              # (data["FlightDate"] == departure_date)
          ]

          # If no flights are found, return a message
          if results.empty:
              return f"There are no flights from {origin} to {destination} ."

          # Format the results into a readable string
          formatted_results = results.to_string(index=False)
          return f"Available flights:\n{formatted_results}"

    except FileNotFoundError:
          return "Flight database file not found. Please ensure the file exists at the specified path."
    except Exception as e:
          return f"An error occurred while searching for flights: {e}"

u("durango","Denver")

'There are no flights from durango to Denver .'